In [1]:
import re 
import tweepy 
from tweepy import OAuthHandler 
from textblob import TextBlob 

In [2]:
class TwitterClient(object): 
	''' 
	Generic Twitter Class for sentiment analysis. 
	'''
	def __init__(self): 
		''' 
		Class constructor or initialization method. 
		'''
		# keys and tokens from the Twitter Dev Console 
		consumer_key = 'qTIVL4BRkIsP1u4K3CiG7viku'
		consumer_secret = 'lj4i87YmUerOeGZ0RVjWfQfduJ95bE58i2fdZAAHzddkzWfp0i'
		access_token = '725621133567905792-FzzWCsYANNpxgQkkwQpxtQg8pau2aqg'
		access_token_secret = 'jnRTAYRboQOT1V50r7tGojzP9uM0BuYiiY2WxPfgSaHAh'

		# attempt authentication 
		try: 
			# create OAuthHandler object 
			self.auth = OAuthHandler(consumer_key, consumer_secret) 
			# set access token and secret 
			self.auth.set_access_token(access_token, access_token_secret) 
			# create tweepy API object to fetch tweets 
			self.api = tweepy.API(self.auth) 
		except: 
			print("Error: Authentication Failed") 

	def clean_tweet(self, tweet): 
		''' 
		Utility function to clean tweet text by removing links, special characters 
		using simple regex statements. 
		'''
		return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split()) 

	def get_tweet_sentiment(self, tweet): 
		''' 
		Utility function to classify sentiment of passed tweet 
		using textblob's sentiment method 
		'''
		# create TextBlob object of passed tweet text 
		analysis = TextBlob(self.clean_tweet(tweet)) 
		# set sentiment 
		if analysis.sentiment.polarity > 0: 
			return 'positive'
		elif analysis.sentiment.polarity == 0: 
			return 'neutral'
		else: 
			return 'negative'

	def get_tweets(self, query, count = 10): 
		''' 
		Main function to fetch tweets and parse them. 
		'''
		# empty list to store parsed tweets 
		tweets = [] 

		try: 
			# call twitter api to fetch tweets 
			fetched_tweets = self.api.search(q = query, count = count) 

			# parsing tweets one by one 
			for tweet in fetched_tweets: 
				# empty dictionary to store required params of a tweet 
				parsed_tweet = {} 

				# saving text of tweet 
				parsed_tweet['text'] = tweet.text 
				# saving sentiment of tweet 
				parsed_tweet['sentiment'] = self.get_tweet_sentiment(tweet.text) 

				# appending parsed tweet to tweets list 
				if tweet.retweet_count > 0: 
					# if tweet has retweets, ensure that it is appended only once 
					if parsed_tweet not in tweets: 
						tweets.append(parsed_tweet) 
				else: 
					tweets.append(parsed_tweet) 

			# return parsed tweets 
			return tweets 

		except tweepy.TweepError as e: 
			# print error (if any) 
			print("Error : " + str(e)) 

In [10]:
# creating object of TwitterClient Class 
api = TwitterClient() 
# calling function to get tweets 
tweets = api.get_tweets(query = 'Jio', count = 500) 

In [11]:
# picking positive tweets from tweets 
ptweets = [tweet for tweet in tweets if tweet['sentiment'] == 'positive'] 
# percentage of positive tweets 
print("Positive tweets percentage: {} %".format(100*len(ptweets)/len(tweets))) 
# picking negative tweets from tweets 
ntweets = [tweet for tweet in tweets if tweet['sentiment'] == 'negative'] 
# percentage of negative tweets 
print("Negative tweets percentage: {} %".format(100*len(ntweets)/len(tweets))) 
# percentage of neutral tweets 
print("Neutral tweets percentage: {} % \ ".format(100*((len(tweets) - len(ntweets) - len(ptweets)))/len(tweets))) 

Positive tweets percentage: 36.8421052631579 %
Negative tweets percentage: 17.894736842105264 %
Neutral tweets percentage: 45.26315789473684 % \ 


In [6]:
# printing first 5 positive tweets 
print("\n\nPositive tweets:") 
for tweet in ptweets[:10]: 
	print(tweet['text']) 

# printing first 5 negative tweets 
print("\n\nNegative tweets:") 
for tweet in ntweets[:10]: 
	print(tweet['text']) 



Positive tweets:
RT @sashachavkin: Major #ImplantFiles impact in India: Health Ministry says it will bring “all medical devices” under its laws for medicine…
RT @airnewsalerts: PM Narendra Modi, released 4 cultural videos today in Delhi marking the #150YrsofMahatma Gandhi.

The event was attended…
RT @OpIndia_com: PM Modi was gracious in congratulating me, economic slowdown not India specific: Esther Duflo on India, PM Modi and the Ec…
RT @FriedrichPieter: @mehdirhasan For a deep dive into #TulsiGabbard’s alliance with far right Narendra Modi and the  anti-Muslim BJP/RSS -…
RT @srivatsayb: "Was she spied upon by IB? Was CJI kept in the dark about her being kept under surveillance? How many other judges are bein…
RT @cjwerleman: Muslims in Kashmir may find themselves in a Xinjiang-style dystopia thanks to Chinese technology and Narendra Modi’s Hindu…
Actor Aamir Khan: First and foremost, I want to appreciate PM Narendra Modi for thinking about this effort (further… https://t.co/zpJzAH